# Appendix

## Imports

In [ ]:
import dolfin
import numpy

from scipy.interpolate import interp1d

import dolfin_mech                    as dmech
import matplotlib.pyplot              as plt
import micro_poro_structure_generator as gen

## Importing experimental data

In [ ]:
smith_PV_inflation_gamma_0 = numpy.load('smith_PV_inflation_gamma_0.npy')
smith_PV_deflation_gamma_0 = numpy.load('smith_PV_deflation_gamma_0.npy')

## Defining geometry

In [ ]:
seeds_filename = "appen.dat"
mesh_filebasename = "appen-mesh"

domain_y = 0.1 * 0.8
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.012 * 0.8

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
dV = dolfin.Measure("dx",domain=mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V = (xmax - xmin)*(ymax - ymin)
VS0 = dolfin.assemble(dolfin.Constant(1) * dV)
Vf0 = V - VS0

In [ ]:
mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
dV = dolfin.Measure("dx",domain=mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V = (xmax - xmin)*(ymax - ymin)
VS0 = dolfin.assemble(dolfin.Constant(1) * dV)
Vf0 = V - VS0

## Loading 

In [ ]:
qois_filename = "appen-qois.dat"
res_basename = "appen"

load_params = {}
load_params["pf"] = 4
load_params["sigma_bar_00"] = 0.0
load_params["sigma_bar_11"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0

## Identifier function

In [ ]:
params_dimless = [0, 0,  0, 0]
params_initial = [0.08855929243285596, 0.011039510924095856, 0.6281487879627474, 3.409513378002055]
exp_data = numpy.vstack((smith_PV_deflation_gamma_0, smith_PV_inflation_gamma_0))
p_exp = exp_data[:, 0]
v_exp = exp_data[:, 1]

bnds = [(-0.9, 10), (-0.8, 10), (-0.2, 0.2), (-0.3, 4)]
asym_slope = 0.1 * Vf0

def J_cost(params_dimless):
    par = [params_initial[0]*(1 + params_dimless[0]), params_initial[1]*(1 + params_dimless[1]), params_initial[2]*(1 + params_dimless[2]), params_initial[3]*(1 + params_dimless[3])]
    print(par)

    mat_params = {"model":"exponentialneoHookean", "parameters":{"beta1":par[0], "beta2":par[1], "beta3":par[2], "beta4":100*par[0], "alpha":par[3]}} 


    dmech.run_HollowBox_MicroPoroHyperelasticity(
        dim=2,
        mesh=mesh,
        mat_params=mat_params,
        load_params=load_params,
        step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.005, "dt_max":0.1},
        res_basename=res_basename,
        write_qois_limited_precision=False,
        verbose=1
    )

    qois_vals = numpy.loadtxt(qois_filename)
    qois_name_list = open(qois_filename).readline().split()
    pf_lst = qois_vals[:, qois_name_list.index("p_f") - 1]*10.20
    vf_lst = qois_vals[:, qois_name_list.index("vf") - 1]

    for i in range(1, len(vf_lst)):
        slope = (vf_lst[i] - vf_lst[i - 1])/(pf_lst[i] - pf_lst[i - 1])
        if slope < asym_slope:
            break

    vf_asym = vf_lst[i]
    vf_lst = [vf_/vf_asym *100 for vf_ in vf_lst]

    model_interpolator = interp1d(pf_lst, vf_lst, kind='cubic') 

    JC = 0
    for i in range(len(exp_data)):
        JC += (v_exp[i] - model_interpolator(p_exp[i]))**2

    print("JC: " +str(JC))
    return JC

## Eigenvalues and Eigenvectors 

In [ ]:
params_dimless = [0, 0, 0, 0]
h = 1e-6
cost_0 =  J_cost([params_dimless[0], params_dimless[1], params_dimless[2], params_dimless[3]])
H_11 = (J_cost([params_dimless[0]+h, params_dimless[1], params_dimless[2], params_dimless[3]]) - 2 * cost_0 + J_cost([params_dimless[0]-h, params_dimless[1], params_dimless[2], params_dimless[3]]))/h**2
H_22 = (J_cost([params_dimless[0], params_dimless[1]+h, params_dimless[2], params_dimless[3]]) - 2 * cost_0 + J_cost([params_dimless[0], params_dimless[1]-h, params_dimless[2], params_dimless[3]]))/h**2
H_33 = (J_cost([params_dimless[0], params_dimless[1], params_dimless[2]+h, params_dimless[3]]) - 2 * cost_0 + J_cost([params_dimless[0], params_dimless[1], params_dimless[2]-h, params_dimless[3]]))/h**2
H_44 = (J_cost([params_dimless[0], params_dimless[1], params_dimless[2], params_dimless[3]+h]) - 2 * cost_0 + J_cost([params_dimless[0], params_dimless[1], params_dimless[2], params_dimless[3]-h]))/h**2
H_12 = (J_cost([params_dimless[0]+h/2, params_dimless[1]+h/2, params_dimless[2], params_dimless[3]]) - J_cost([params_dimless[0]-h/2, params_dimless[1]+h/2, params_dimless[2], params_dimless[3]]) - J_cost([params_dimless[0]+h/2, params_dimless[1]-h/2, params_dimless[2], params_dimless[3]]) + J_cost([params_dimless[0]-h/2, params_dimless[1]-h/2, params_dimless[2], params_dimless[3]]))/h**2
H_13 = (J_cost([params_dimless[0]+h/2, params_dimless[1], params_dimless[2]+h/2, params_dimless[3]]) - J_cost([params_dimless[0]-h/2, params_dimless[1], params_dimless[2]+h/2, params_dimless[3]]) - J_cost([params_dimless[0]+h/2, params_dimless[1], params_dimless[2]-h/2, params_dimless[3]]) + J_cost([params_dimless[0]-h/2, params_dimless[1], params_dimless[2]-h/2, params_dimless[3]]))/h**2
H_14 = (J_cost([params_dimless[0]+h/2, params_dimless[1], params_dimless[2], params_dimless[3]+h/2]) - J_cost([params_dimless[0]-h/2, params_dimless[1], params_dimless[2], params_dimless[3]+h/2]) - J_cost([params_dimless[0]+h/2, params_dimless[1], params_dimless[2], params_dimless[3]-h/2]) + J_cost([params_dimless[0]-h/2, params_dimless[1], params_dimless[2], params_dimless[3]-h/2]))/h**2
H_23 = (J_cost([params_dimless[0], params_dimless[1]+h/2, params_dimless[2]+h/2, params_dimless[3]]) - J_cost([params_dimless[0], params_dimless[1]-h/2, params_dimless[2]+h/2, params_dimless[3]]) - J_cost([params_dimless[0], params_dimless[1]+h/2, params_dimless[2]-h/2, params_dimless[3]]) + J_cost([params_dimless[0], params_dimless[1]-h/2, params_dimless[2]-h/2, params_dimless[3]]))/h**2
H_24 = (J_cost([params_dimless[0], params_dimless[1]+h/2, params_dimless[2], params_dimless[3]+h/2]) - J_cost([params_dimless[0], params_dimless[1]-h/2, params_dimless[2], params_dimless[3]+h/2]) - J_cost([params_dimless[0], params_dimless[1]+h/2, params_dimless[2], params_dimless[3]-h/2]) + J_cost([params_dimless[0], params_dimless[1]-h/2, params_dimless[2], params_dimless[3]-h/2]))/h**2
H_34 = (J_cost([params_dimless[0], params_dimless[1], params_dimless[2]+h/2, params_dimless[3]+h/2]) - J_cost([params_dimless[0], params_dimless[1], params_dimless[2]-h/2, params_dimless[3]+h/2]) - J_cost([params_dimless[0], params_dimless[1], params_dimless[2]+h/2, params_dimless[3]-h/2]) + J_cost([params_dimless[0], params_dimless[1], params_dimless[2]-h/2, params_dimless[3]-h/2]))/h**2
H_21 = H_12
H_31 = H_13
H_32 = H_23
H_41 = H_14
H_42 = H_24
H_43 = H_34
Hess = numpy.matrix([[float(H_11), float(H_12), float(H_13), float(H_14)],
                  [float(H_21), float(H_22), float(H_23), float(H_24)],
                  [float(H_31), float(H_32), float(H_33), float(H_34)],
                  [float(H_41), float(H_42), float(H_43), float(H_44)]])
# print(Hess)
print(numpy.linalg.eig(Hess))

In [ ]:
h2 = 0.1
C1_C2_lim=numpy.linspace(float(params_dimless[0] - h2),float(params_dimless[0] + h2),11)
C2_C1_lim=numpy.linspace(float(params_dimless[1] - h2),float(params_dimless[1] + h2),11)
 
X, Y = numpy.meshgrid(C1_C2_lim,C2_C1_lim)
Z_C1_C2= numpy.zeros((len(C1_C2_lim), len(C2_C1_lim)))
for i in range (len(C1_C2_lim)):
    for j in range (len(C2_C1_lim)):
        Z_C1_C2[i, j] = J_cost([C1_C2_lim[i], C2_C1_lim[j], params_dimless[2], params_dimless[3]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C1_C2, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\beta_1$', fontsize=12)
ax.set_ylabel(r'$\beta_2$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('beta1_beta2.pdf',bbox_inches='tight')

In [ ]:
C1_C3_lim=numpy.linspace(float(params_dimless[0] - h2),float(params_dimless[0] + h2),11)
C3_C1_lim=numpy.linspace(float(params_dimless[2] - h2),float(params_dimless[2] + h2),11)
 
X, Y = numpy.meshgrid(C1_C3_lim,C3_C1_lim)
Z_C1_C3= numpy.zeros((len(C1_C3_lim), len(C3_C1_lim)))
for i in range (len(C1_C3_lim)):
    for j in range (len(C3_C1_lim)):
        Z_C1_C3[i, j] = J_cost([C1_C3_lim[i], params_dimless[1], C3_C1_lim[j], params_dimless[3]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C1_C3, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\beta_1$', fontsize=12)
ax.set_ylabel(r'$\alpha$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('beta1_alpha.pdf',bbox_inches='tight')

In [ ]:
C2_C3_lim=numpy.linspace(float(params_dimless[1] - h2),float(params_dimless[1] + h2),11)
C3_C2_lim=numpy.linspace(float(params_dimless[2] - h2),float(params_dimless[2] + h2),11)
 
X, Y = numpy.meshgrid(C2_C3_lim,C3_C2_lim)
Z_C2_C3= numpy.zeros((len(C2_C3_lim), len(C3_C2_lim)))
for i in range (len(C2_C3_lim)):
    for j in range (len(C3_C2_lim)):
        Z_C2_C3[i, j] = J_cost([params_dimless[0], C2_C3_lim[i], C3_C2_lim[j], params_dimless[3]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C2_C3, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\beta_2$', fontsize=12)
ax.set_ylabel(r'$\alpha$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('beta2_alpha.pdf',bbox_inches='tight')

In [ ]:
C1_C4_lim=numpy.linspace(float(params_dimless[0] - h2),float(params_dimless[0] + h2),11)
C4_C1_lim=numpy.linspace(float(params_dimless[3] - h2),float(params_dimless[3] + h2),11)
 
X, Y = numpy.meshgrid(C1_C4_lim,C4_C1_lim)
Z_C1_C4= numpy.zeros((len(C1_C4_lim), len(C4_C1_lim)))
for i in range (len(C1_C4_lim)):
    for j in range (len(C4_C1_lim)):
        Z_C1_C4[i, j] = J_cost([C1_C4_lim[i], params_dimless[1], params_dimless[2], C4_C1_lim[j]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C1_C4, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\beta_1$', fontsize=12)
ax.set_ylabel(r'$\beta_3$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('beta1_beta3.pdf',bbox_inches='tight')

In [ ]:
C2_C4_lim=numpy.linspace(float(params_dimless[1] - h2),float(params_dimless[1] + h2),11)
C4_C2_lim=numpy.linspace(float(params_dimless[3] - h2),float(params_dimless[3] + h2),11)
 
X, Y = numpy.meshgrid(C2_C4_lim,C4_C2_lim)
Z_C2_C4= numpy.zeros((len(C2_C4_lim), len(C4_C2_lim)))
for i in range (len(C2_C4_lim)):
    for j in range (len(C4_C2_lim)):
        Z_C2_C4[i, j] = J_cost([params_dimless[0], C2_C4_lim[i], params_dimless[2], C4_C2_lim[j]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C2_C4, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\beta_2$', fontsize=12)
ax.set_ylabel(r'$\beta_3$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('beta2_beta3.pdf',bbox_inches='tight')

In [ ]:
C3_C4_lim=numpy.linspace(float(params_dimless[2] - h2),float(params_dimless[2] + h2),11)
C4_C3_lim=numpy.linspace(float(params_dimless[3] - h2),float(params_dimless[3] + h2),11)
 
X, Y = numpy.meshgrid(C3_C4_lim,C4_C3_lim)
Z_C3_C4= numpy.zeros((len(C3_C4_lim), len(C4_C3_lim)))
for i in range (len(C3_C4_lim)):
    for j in range (len(C4_C3_lim)):
        Z_C3_C4[i, j] = J_cost([params_dimless[0], params_dimless[1], C3_C4_lim[i], C4_C3_lim[j]])

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z_C3_C4, cmap="plasma", linewidth=0, antialiased=False, alpha=0.5)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

ax.set_xlabel(r'$\alpha$', fontsize=12)
ax.set_ylabel(r'$\beta_3$', fontsize=12)
ax.set_zlabel('Cost function', fontsize=12)
plt.savefig('alpha_beta3.pdf', bbox_inches='tight')